In [1]:
import pandas as pd
import numpy as np
import re
import warnings
import random
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity
import surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

### 데이터 전처리 과정

In [2]:
book_df=pd.read_csv('Books.csv',low_memory=False)
rating_df=pd.read_csv('Ratings.csv')
user_df=pd.read_csv('Users.csv')

In [3]:
book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
rating_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
user_df['User-ID'].equals(rating_df['User-ID'])

False

In [7]:
book_df['ISBN'].equals(rating_df['ISBN'])

False

In [8]:
book_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [9]:
book_df.loc[book_df['Book-Author'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...


In [10]:
book_df.loc[book_df['Publisher'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...


In [11]:
book_df.loc[book_df['Image-URL-L'].isnull(),:]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


In [12]:
book_df['Book-Author']= book_df['Book-Author'].fillna('Other')
book_df['Publisher']= book_df['Publisher'].fillna('Other')
book_df['Image-URL-L']= book_df['Image-URL-L'].fillna('no_image.jpg')

In [13]:
## 출판연도 항목 처리
book_df['Year-Of-Publication'].unique()

array(['2002', '2001', '1991', '1999', '2000', '1993', '1996', '1988',
       '2004', '1998', '1994', '2003', '1997', '1983', '1979', '1995',
       '1982', '1985', '1992', '1986', '1978', '1980', '1952', '1987',
       '1990', '1981', '1989', '1984', '0', '1968', '1961', '1958',
       '1974', '1976', '1971', '1977', '1975', '1965', '1941', '1970',
       '1962', '1973', '1972', '1960', '1966', '1920', '1956', '1959',
       '1953', '1951', '1942', '1963', '1964', '1969', '1954', '1950',
       '1967', '2005', '1957', '1940', '1937', '1955', '1946', '1936',
       '1930', '2011', '1925', '1948', '1943', '1947', '1945', '1923',
       '2020', '1939', '1926', '1938', '2030', '1911', '1904', '1949',
       '1932', '1928', '1929', '1927', '1931', '1914', '2050', '1934',
       '1910', '1933', '1902', '1924', '1921', '1900', '2038', '2026',
       '1944', '1917', '1901', '2010', '1908', '1906', '1935', '1806',
       '2021', '2012', '2006', 'DK Publishing Inc', 'Gallimard', '1909',
       

In [14]:
book_df.at[209538 ,'Publisher'] = 'DK Publishing Inc'
book_df.at[209538 ,'Year-Of-Publication'] = 2000
book_df.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
book_df.at[209538 ,'Book-Author'] = 'Michael Teitelbaum'

book_df.at[221678 ,'Publisher'] = 'DK Publishing Inc'
book_df.at[221678 ,'Year-Of-Publication'] = 2000
book_df.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic book_df Come to Life (Level 4: Proficient Readers)'
book_df.at[209538 ,'Book-Author'] = 'James Buckley'

book_df.at[220731 ,'Publisher'] = 'Gallimard'
book_df.at[220731 ,'Year-Of-Publication'] = '2003'
book_df.at[209538 ,'Book-Title'] = 'Peuple du ciel - Suivi de Les bergers '
book_df.at[209538 ,'Book-Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [15]:
rating_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [16]:
dataset = pd.merge(book_df, rating_df, on='ISBN', how='inner')
dataset = pd.merge(dataset, user_df, on='User-ID', how='inner')
dataset.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0,"stockton, california, usa",18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5,"timmins, ontario, canada",NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,8,0,"timmins, ontario, canada",NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,8,0,"timmins, ontario, canada",NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,8,0,"timmins, ontario, canada",NaN


In [17]:
# 전처리된 데이터 항목들
real_rating_dataset = dataset[dataset['Book-Rating'] != 0]
real_rating_dataset = real_rating_dataset.reset_index(drop = True)

In [18]:
real_rating_dataset.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5,"timmins, ontario, canada",NaN
1,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,8,5,"timmins, ontario, canada",NaN
2,0887841740,The Middle Stories,Sheila Heti,2004,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,http://images.amazon.com/images/P/0887841740.0...,http://images.amazon.com/images/P/0887841740.0...,8,5,"timmins, ontario, canada",NaN
3,1552041778,Jane Doe,R. J. Kaiser,1999,Mira Books,http://images.amazon.com/images/P/1552041778.0...,http://images.amazon.com/images/P/1552041778.0...,http://images.amazon.com/images/P/1552041778.0...,8,5,"timmins, ontario, canada",NaN
4,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,http://images.amazon.com/images/P/1567407781.0...,http://images.amazon.com/images/P/1567407781.0...,http://images.amazon.com/images/P/1567407781.0...,8,6,"timmins, ontario, canada",NaN


In [19]:
df = real_rating_dataset.copy()
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(columns=["Year-Of-Publication","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)
df["Book-Title"]=df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
df.head()

,ISBN,Book-Title,Book-Author,Publisher,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0786868716,The Five People You Meet in Heaven,Mitch Albom,Hyperion,http://images.amazon.com/images/P/0786868716.0...,11400,9,"ottawa, ontario, canada",49.0
1,0151008116,Life of Pi,Yann Martel,Harcourt,http://images.amazon.com/images/P/0151008116.0...,11400,6,"ottawa, ontario, canada",49.0
2,0312195516,The Red Tent Bestselling Backlist,Anita Diamant,Picador USA,http://images.amazon.com/images/P/0312195516.0...,11400,7,"ottawa, ontario, canada",49.0
3,0316789089,The Pilot s Wife A Novel Tag Author of the Wei...,Anita Shreve,"Little, Brown",http://images.amazon.com/images/P/0316789089.0...,11400,7,"ottawa, ontario, canada",49.0
4,0743418174,Good in Bed,Jennifer Weiner,Washington Square Press,http://images.amazon.com/images/P/0743418174.0...,11400,8,"ottawa, ontario, canada",49.0


### svd을 이용한 점수 예측 프로그램 설정

In [20]:
reader=Reader(rating_scale=(1,10))

In [21]:
data=Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader=reader)
data

In [22]:
svd = SVD(random_state=0)

In [23]:
cross_validate(svd, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5984  1.6153  1.6132  1.6119  1.6099  1.6097  0.0060  
MAE (testset)     1.2317  1.2421  1.2404  1.2377  1.2391  1.2382  0.0036  
Fit time          3.58    3.87    3.80    3.64    3.62    3.70    0.11    
Test time         0.38    0.51    0.50    0.51    0.37    0.45    0.06    


{'test_rmse': array([1.59835427, 1.6153164 , 1.6131938 , 1.61189222, 1.6099146 ]),
 'test_mae': array([1.23170765, 1.24213377, 1.24037322, 1.23767181, 1.23905664]),
 'fit_time': (3.5803754329681396,
  3.8695783615112305,
  3.803361415863037,
  3.6393637657165527,
  3.624608278274536),
 'test_time': (0.3750803470611572,
  0.5107300281524658,
  0.49991512298583984,
  0.5097155570983887,
  0.37496066093444824)}

In [24]:
# 전체 데이터 학습 시키기
trainset = data.build_full_trainset()
svd.fit(trainset)

In [87]:
# 원하는 유저의 평가 항목 검색
df[df['User-ID']==11400]
print('유저 리스트 입니다 다른 항목을 보려면 아무키나 누르세요 멈추 시려면 y를 누르세요')
ma='a'
while(ma!='y'):
    s=random.randint(1,len(df['User-ID']))
    print(df['User-ID'][s:s+10,])
    ma=input()

id_1=input('원하는 유저의 아이디를 입력하세요: ')
isbn=df.loc[df['User-ID']==int(id_1),'ISBN']
ti=df.loc[df['User-ID']==int(id_1),'Book-Title']
print('\n해당 유저가 평가한 책들의 ISBN과 제목입니다\n')
print(isbn, ti)


유저 리스트 입니다 다른 항목을 보려면 아무키나 누르세요 멈추 시려면 y를 누르세요
86233    207246
86234    207246
86235    207246
86236    207246
86237    207246
86238    207246
86239    207246
86240    207246
86241    207246
86242    207246
Name: User-ID, dtype: int64

195801    249598
195802    249598
195803    249598
195804     37597
195805     37597
195806     37597
195807     37597
195808     37597
195809     37597
195810     37597
Name: User-ID, dtype: int64

217544      1775
217545     57037
217546     57037
217547    140635
217548    140635
217549    151777
217550    151777
217551    151777
217552     76569
217553    254038
Name: User-ID, dtype: int64

11587    125519
11588    125519
11589    125519
11590    125519
11591    125519
11592    125519
11593    125519
11594    125519
11595    125519
11596    125519
Name: User-ID, dtype: int64

197142    151405
197143    151405
197144    151405
197145    151405
197146    151405
197147    189066
197148    189066
197149    189066
197150    189066
197151    189066
Name: U

In [95]:
ISBN=input('평가를 원하시는 책의 isbn을 입력해주세요:')
print(f'평가 점수: {svd.predict(int(id_1),ISBN).est:.2f}')

평가를 원하시는 책의 isbn을 입력해주세요:0380820447
7.99


In [27]:
df.loc[0, 'Book-Title']

'The Five People You Meet in Heaven'

### cosine 유사도의 머신러닝 기법을 이용한 유저 기반 평가 프로그램


In [28]:
# 다른 방식을 위한 데이터 전처리
df2=real_rating_dataset.copy()
df2.dropna(inplace=True)
df2.reset_index(drop=True,inplace=True)
df2.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
df2.drop(index=df2[df2["Book-Rating"]==0].index,inplace=True)
df2["Book-Title"]=df2["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
df2.head()

,Book-Title,Book-Author,Publisher,Image-URL-L,User-ID,Book-Rating,Location,Age
0,The Five People You Meet in Heaven,Mitch Albom,Hyperion,http://images.amazon.com/images/P/0786868716.0...,11400,9,"ottawa, ontario, canada",49.0
1,Life of Pi,Yann Martel,Harcourt,http://images.amazon.com/images/P/0151008116.0...,11400,6,"ottawa, ontario, canada",49.0
2,The Red Tent Bestselling Backlist,Anita Diamant,Picador USA,http://images.amazon.com/images/P/0312195516.0...,11400,7,"ottawa, ontario, canada",49.0
3,The Pilot s Wife A Novel Tag Author of the Wei...,Anita Shreve,"Little, Brown",http://images.amazon.com/images/P/0316789089.0...,11400,7,"ottawa, ontario, canada",49.0
4,Good in Bed,Jennifer Weiner,Washington Square Press,http://images.amazon.com/images/P/0743418174.0...,11400,8,"ottawa, ontario, canada",49.0


In [29]:
new_df=df2[df2['User-ID'].map(df2['User-ID'].value_counts()) > 200]  # Drop users who vote less than 200 times.
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)
new_df["User-ID"].values
l=new_df["User-ID"].values
idxa=random.randint(1,len(l))

In [30]:
def users_choice(id):
    
    users_fav=new_df[new_df["User-ID"]==id].sort_values(["Book-Rating"],ascending=False)[0:5]
    return users_fav

In [31]:
def user_based(new_df,id):
    if id not in new_df["User-ID"].values:
        print("❌ User NOT FOUND ❌")
        
        
    else:
        index=np.where(users_pivot.index==id)[0][0]
        similarity=cosine_similarity(users_pivot)
        similar_users=list(enumerate(similarity[index]))
        similar_users = sorted(similar_users,key = lambda x:x[1],reverse=True)[0:5]
    
        user_rec=[]
    
        for i in similar_users:
                data=df[df["User-ID"]==users_pivot.index[i[0]]]
                user_rec.extend(list(data.drop_duplicates("User-ID")["User-ID"].values))
        
    return user_rec

In [32]:
def common(new_df,user,user_id):
    x=new_df[new_df["User-ID"]==user_id]
    recommend_books=[]
    user=list(user)
    for i in user:
        y=new_df[(new_df["User-ID"]==i)]
        books=y.loc[~y["Book-Title"].isin(x["Book-Title"]),:]
        books=books.sort_values(["Book-Rating"],ascending=False)[0:5]
        recommend_books.extend(books["Book-Title"].values)
        
    return recommend_books[0:5]

In [33]:
user_id=l[idxa]
user_choice_df=pd.DataFrame(users_choice(user_id))
user_favorite=users_choice(user_id)
n=len(user_choice_df["Book-Title"].values)
print("USER: {} ".format(user_id))
print()
    
print("랜덤 선택된 유저가 선호하는 책")
print()

for i in range(n):
    b_name=new_df.loc[new_df["Book-Title"]==user_choice_df["Book-Title"].tolist()[i],"Book-Title"][:1].values[0]
    b_url=new_df.loc[new_df["Book-Title"]==user_choice_df["Book-Title"].tolist()[i],"Image-URL-L"][:1].values[0]
    print(f'제목 {b_name}')
    print(f'표지: {b_url}\n')
    
print('\n\n')
user_based_rec=user_based(new_df,user_id)
books_for_user=common(new_df,user_based_rec,user_id)
books_for_userDF=pd.DataFrame(books_for_user,columns=["Book-Title"])

print("선호도 기반한 추천 책")
print()

for i in range(5):
    b_name=new_df.loc[new_df["Book-Title"]==books_for_userDF["Book-Title"].tolist()[i],"Book-Title"][:1].values[0]
    b_url=new_df.loc[new_df["Book-Title"]==books_for_userDF["Book-Title"].tolist()[i],"Image-URL-L"][:1].values[0]
    print(f'제목 {b_name}')
    print(f'표지: {b_url}\n')


USER: 16795 

랜덤 선택된 유저가 선호하는 책

제목 The Pull of the Moon
표지: http://images.amazon.com/images/P/0515120898.01.LZZZZZZZ.jpg

제목 Skipping Christmas
표지: http://images.amazon.com/images/P/0385505833.01.LZZZZZZZ.jpg

제목 Everything s Eventual 14 Dark Tales
표지: http://images.amazon.com/images/P/0743235150.01.LZZZZZZZ.jpg

제목 No Second Chance
표지: http://images.amazon.com/images/P/0525947299.01.LZZZZZZZ.jpg

제목 Ordinary Life Stories
표지: http://images.amazon.com/images/P/0679437460.01.LZZZZZZZ.jpg




선호도 기반한 추천 책

제목 Princess A True Story of Life Behind the Veil in Saudi Arabia
표지: http://images.amazon.com/images/P/0380719185.01.LZZZZZZZ.jpg

제목 Love Tangle
표지: http://images.amazon.com/images/P/0821736051.01.LZZZZZZZ.jpg

제목 The Magic of You Malory Novels Paperback
표지: http://images.amazon.com/images/P/0380756293.01.LZZZZZZZ.jpg

제목 Tuesdays with Morrie An Old Man a Young Man and Life s Greatest Lesson
표지: http://images.amazon.com/images/P/0385484518.01.LZZZZZZZ.jpg

제목 FLOWERS IN ATTIC Dollange